In [1]:
# In this notebook, you learn:
#
# 1) How to use pytorch to train models on GPU?

In [2]:
# Resources:
#
# 1) https://youtu.be/6stDhEA0wFQ?si=rkc0iKKRxWnaYbYo
#       -- Gives a very high level overview of why we use GPU's?
# 2) https://youtu.be/Bs1mdHZiAS8?si=0SpkfO3POIuffsv3
#       -- Shows how to create tensors and modules on GPU in pytorch.
# 3) https://wandb.ai/ayush-thakur/dl-question-bank/reports/How-To-Check-If-PyTorch-Is-Using-The-GPU--VmlldzoyMDQ0NTU
#       -- Shows very basic commands to check about the GPUs in your system.
# 4) https://www.run.ai/guides/gpu-deep-learning/pytorch-gpu
#       -- Gives a bit more details about how GPUs are used in pytorch.

In [15]:
from torch import nn, Tensor

import torch

In [12]:
DEVICE_CUDA = "cuda"
DEVICE_CPU = "cpu"

In [4]:
# This is how you can check if GPU is available (on your machine) or not.
torch.cuda.is_available()

True

In [6]:
# This prints the number of GPU's available on your machine.
torch.cuda.device_count()

1

In [7]:
# This prints the name of the GPU available on your machine.
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 4090 Laptop GPU'

In [13]:
def get_device():
    return torch.device(DEVICE_CUDA if torch.cuda.is_available() else DEVICE_CPU)

In [9]:
# By default, the tensors are created on CPU. We have to explicitly create tensors on GPU or move them to GPU.
t1 = torch.tensor(data=[1, 2, 3], dtype=torch.float32)
print("shape of t1: ", t1.shape)
print("t1: ", t1)
print("Is t1 on GPU: ", t1.is_cuda)

shape of t1:  torch.Size([3])
t1:  tensor([1., 2., 3.])
Is t1 on GPU:  False


In [11]:
t2 = t1.to(DEVICE_CUDA)
print("shape of t2: ", t2.shape)
print("t2: ", t2)
print("Is t2 on GPU: ", t2.is_cuda)
print("-" * 150)
print("shape of t1: ", t1.shape)
print("t1: ", t1)
print("Is t1 on GPU: ", t1.is_cuda)
print("-" * 150)
# Apparently, sometimes the data is not explicitly copied to GPU but the pointer on the GPU just points to the
# data on the CPU. This is called as "sharing memory location". I need to dig deep into the workings of GPU to
# understand this better.
if t1.data_ptr() == t2.data_ptr():
    print("t1 and t2 are sharing the same memory location.")
else:
    print("t1 and t2 are not sharing the same memory location.")

shape of t2:  torch.Size([3])
t2:  tensor([1., 2., 3.], device='cuda:0')
Is t2 on GPU:  True
------------------------------------------------------------------------------------------------------------------------------------------------------
shape of t1:  torch.Size([3])
t1:  tensor([1., 2., 3.])
Is t1 on GPU:  False
------------------------------------------------------------------------------------------------------------------------------------------------------
t1 and t2 are not sharing the same memory location.


In [16]:
# Lets create a simple neural network and train it on GPU.
class SimpleNeuralNetwork(nn.Module):
    def __init__(self):
        super(SimpleNeuralNetwork, self).__init__()
        self.linear_layer_1 = nn.Linear(in_features=3, out_features=10)
        self.relu = nn.ReLU()
        self.linear_layer_2 = nn.Linear(in_features=10, out_features=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input: Tensor) -> Tensor:
        output = self.linear_layer_1(input)
        output = self.relu(output)
        output = self.linear_layer_2(output)
        output = self.sigmoid(output)
        return output

In [17]:
# A network can have multiple trainable parameters. Some of these parameters can be stored on GPU and some on CPU.
# So, there isn't a single way to check if the network is on GPU or not. We have to check for each parameter.
network_1 = SimpleNeuralNetwork()
print(network_1)

SimpleNeuralNetwork(
  (linear_layer_1): Linear(in_features=3, out_features=10, bias=True)
  (relu): ReLU()
  (linear_layer_2): Linear(in_features=10, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [19]:
for name, parameters in network_1.named_parameters():
    print("name: ", name)
    print("Is parameters on GPU: ", parameters.is_cuda)
    print("-" * 150)

name:  linear_layer_1.weight
Is parameters on GPU:  False
------------------------------------------------------------------------------------------------------------------------------------------------------
name:  linear_layer_1.bias
Is parameters on GPU:  False
------------------------------------------------------------------------------------------------------------------------------------------------------
name:  linear_layer_2.weight
Is parameters on GPU:  False
------------------------------------------------------------------------------------------------------------------------------------------------------
name:  linear_layer_2.bias
Is parameters on GPU:  False
------------------------------------------------------------------------------------------------------------------------------------------------------


In [21]:
# Just to confirm that the device is being returned as 'gpu' correctly on my machine. This might return 'cpu' on your\
# machine if you don't have a GPU.
print(get_device())
print("-" * 150)
# All the parameters in the network can be moved to GPU using a single call.
network_1.to(get_device())
# Notice that all the parameters are now on GPU.
for name, parameters in network_1.named_parameters():
    print("name: ", name)
    print("Is parameters on GPU: ", parameters.is_cuda)
    print("-" * 150)

cuda
------------------------------------------------------------------------------------------------------------------------------------------------------
name:  linear_layer_1.weight
Is parameters on GPU:  True
------------------------------------------------------------------------------------------------------------------------------------------------------
name:  linear_layer_1.bias
Is parameters on GPU:  True
------------------------------------------------------------------------------------------------------------------------------------------------------
name:  linear_layer_2.weight
Is parameters on GPU:  True
------------------------------------------------------------------------------------------------------------------------------------------------------
name:  linear_layer_2.bias
Is parameters on GPU:  True
------------------------------------------------------------------------------------------------------------------------------------------------------


In [25]:
# Lets try to run the forward pass on CPU.
input_1 = torch.tensor(data=[[1, 2, 3], [4, 5, 6]], dtype=torch.float32, device="cpu")
print("shape: ", input.shape)
print("input: \n", input)

shape:  torch.Size([2, 3])
input: 
 tensor([[1., 2., 3.],
        [4., 5., 6.]])


In [26]:
# This threw an error as expected because the input is on CPU and the network is on GPU. Both the input and the network
# should be on the same device.
output_1 = network_1(input_1)
print(output_1)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [27]:
# Lets try to create the input on GPU and then run the forward pass.
input_2 = torch.tensor(data=[[1, 2, 3], [4, 5, 6]], dtype=torch.float32, device=get_device())
print("shape: ", input_2.shape)
print("input: \n", input_2)

shape:  torch.Size([2, 3])
input: 
 tensor([[1., 2., 3.],
        [4., 5., 6.]], device='cuda:0')


In [28]:
# Successfully runs the forward pass as expected.
output_2 = network_1(input_2)
print("shape: ", output_2.shape)
print("output: \n", output_2)

shape:  torch.Size([2, 1])
output: 
 tensor([[0.3091],
        [0.1033]], device='cuda:0', grad_fn=<SigmoidBackward0>)
